In [1]:
import pickle
import math
import os
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
from mamba_ssm import Mamba2
import random
random.seed(690)
np.random.seed(690)
torch.manual_seed(689)
torch.cuda.manual_seed(609)
torch.cuda.manual_seed_all(679)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [2]:

def collate_fn(batch):
    sequences, labels = zip(*batch)
    padded_sequences = nn.utils.rnn.pad_sequence(sequences, batch_first=True)
    return padded_sequences, torch.stack(labels) if isinstance(labels[0], torch.Tensor) else labels

class Mamba2Classifier(nn.Module):
    def __init__(self, input_size, num_classes, d_state=64, d_conv=4, expand=2):
        super(Mamba2Classifier, self).__init__()
        self.mamba = Mamba2(
            d_model=input_size,
            d_state=d_state,
            d_conv=d_conv,
            expand=expand
        )
        self.classifier = nn.Linear(input_size, num_classes)

    def forward(self, x):
        x = self.mamba(x)
        x = x.mean(dim=1)
        return self.classifier(x)

class SequenceDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        self.data = [(os.path.join(root_dir, cls, file), self.class_to_idx[cls])
                     for cls in self.classes
                     for file in os.listdir(os.path.join(root_dir, cls))
                     if file.endswith('.npy')]

    def __len__(self):
        
        return len(self.data)

    def __getitem__(self, idx):
        file_path, label = self.data[idx]
        sequence = np.load(file_path, allow_pickle=True)
        sequence = (sequence - np.mean(sequence)) / np.std(sequence)
        return torch.tensor(sequence, dtype=torch.float32).squeeze(), torch.tensor(label, dtype=torch.long)

class EvaluationDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.file_paths = [os.path.join(root_dir, file) for file in os.listdir(root_dir)]

    def __len__(self):
        print(len(self.file_paths))
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        sequence = np.load(file_path, allow_pickle=True)
        sequence = (sequence - np.mean(sequence)) / np.std(sequence)
        return torch.tensor(sequence, dtype=torch.float32).squeeze(), os.path.basename(file_path)


class EvaluationDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.file_paths = []
        
        print(f"Initializing EvaluationDataset with root_dir: {root_dir}")
        
        if not os.path.exists(root_dir):
            print(f"Error: The directory {root_dir} does not exist.")
            return
        
        for file in os.listdir(root_dir):
            if file.endswith('.npy'):
                full_path = os.path.join(root_dir, file)
                self.file_paths.append(full_path)
        
        print(f"Found {len(self.file_paths)} .npy files in {root_dir}")
        
        if len(self.file_paths) == 0:
            print("Warning: No .npy files found in the directory.")
            print("Contents of the directory:")
            print(os.listdir(root_dir))

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        try:
            sequence = np.load(file_path, allow_pickle=True)
            sequence = (sequence - np.mean(sequence)) / np.std(sequence)
            return torch.tensor(sequence, dtype=torch.float32).squeeze(), os.path.basename(file_path)
        except Exception as e:
            print(f"Error loading file {file_path}: {str(e)}")
            return torch.tensor([]), ""
            
def calculate_metrics(loader, model, device):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            scores = model(x)
            _, predictions = scores.max(1)
            y_true.extend(y.cpu().numpy())
            y_pred.extend(predictions.cpu().numpy())
    return (accuracy_score(y_true, y_pred),
            precision_score(y_true, y_pred, average='weighted'),
            recall_score(y_true, y_pred, average='weighted'),
            f1_score(y_true, y_pred, average='weighted'))

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device):
    model.to(device)
    best_accuracy = 0
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for data, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            data, targets = data.to(device), targets.to(device)
            scores = model(data)
            loss = criterion(scores, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)
        train_metrics = calculate_metrics(train_loader, model, device)
        val_metrics = calculate_metrics(val_loader, model, device)

        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"Train Loss: {avg_train_loss:.4f}")
        print(f"Train: Accuracy: {train_metrics[0]:.4f}, Precision: {train_metrics[1]:.4f}, Recall: {train_metrics[2]:.4f}, F1: {train_metrics[3]:.4f}")
        print(f"Val: Accuracy: {val_metrics[0]:.4f}, Precision: {val_metrics[1]:.4f}, Recall: {val_metrics[2]:.4f}, F1: {val_metrics[3]:.4f}")
        print(f"Learning Rate: {scheduler.get_last_lr()[0]}")

        scheduler.step()

        if val_metrics[0] > best_accuracy:
            best_accuracy = val_metrics[0]
            torch.save(model.state_dict(), 'best_model.pth')
            print(f"New best model saved with accuracy: {best_accuracy:.4f}")

def evaluate_model(model, data_loader, device):
    model.eval()
    all_predictions = []
    all_file_names = []

    with torch.no_grad():
        for data, file_names in tqdm(data_loader, desc="Evaluating"):
            data = data.to(device)
            outputs = model(data)
            _, predictions = torch.max(outputs, 1)
            all_predictions.extend(predictions.cpu().numpy())
            all_file_names.extend(file_names)

    return all_predictions, all_file_names

def save_results_to_csv(file_names, predictions, class_names, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        header = ['frontal_view_video_name'] + class_names
        writer.writerow(header)

        for file_name, pred in zip(file_names, predictions):
            file_name_without_ext = os.path.splitext(file_name)[0]
            row = [file_name_without_ext] + [1 if i == pred else 0 for i in range(len(class_names))]
            writer.writerow(row)

def save_results_to_csv(file_names, predictions, class_names, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        header = ['frontal_view_video_name'] + class_names
        writer.writerow(header)

        for file_name, pred in zip(file_names, predictions):
            file_name_without_ext = os.path.splitext(file_name)[0]
            row = [file_name_without_ext] + [1 if i == pred else 0 for i in range(len(class_names))]
            writer.writerow(row)
def save_results_to_csv(file_names, predictions, class_names, output_file):
       print(f"Attempting to save results to {output_file}")
       print(f"Number of file names: {len(file_names)}")
       print(f"Number of predictions: {len(predictions)}")
       print(f"Class names: {class_names}")
       
       with open(output_file, 'w', newline='') as csvfile:
           writer = csv.writer(csvfile)
           header = ['frontal_view_video_name'] + class_names
           writer.writerow(header)

           for file_name, pred in zip(file_names, predictions):
               file_name_without_ext = os.path.splitext(file_name)[0]
               row = [file_name_without_ext] + [1 if i == pred else 0 for i in range(len(class_names))]
               writer.writerow(row)
       
       print(f"File should have been saved. Does it exist? {os.path.exists(output_file)}")
       if os.path.exists(output_file):
           
           print(f"File path: {os.path.abspath(output_file)}")
           print(f"File size: {os.path.getsize(output_file)} bytes")
def main():
    input_size = 512
    num_classes = 6
    d_state = 32   #32
    d_conv = 4   #4
    expand = 8   #2 was the best
    batch_size = 16
    learning_rate = 0.001
    num_epochs = 20
    train_dataset = SequenceDataset('/workspace/data/VGG16_Training_Features/frontal_view')
    eval_dataset = SequenceDataset('/workspace/data/VGG16_val_features/frontal_view')
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    eval_loader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    model = Mamba2Classifier(input_size, num_classes, d_state, d_conv, expand)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.8)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train_model(model, train_loader, eval_loader, criterion, optimizer, scheduler, num_epochs, device)
    model.load_state_dict(torch.load('best_model.pth'))
    test_dataset = EvaluationDataset('/workspace/data/VGG16_test_features/frontal_view/VGG16_features')
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    predictions, file_names = evaluate_model(model, test_loader, device)

    class_names = ['Left Lane Change', 'Left Turn', 'Right Lane Change', 'Right Turn', 'Slow-Stop', 'Straight']
    save_results_to_csv(file_names, predictions, class_names, '/workspace/mamba2_test_results.csv')

    print("Testing completed. Results saved to mamba2_test_results.csv")

    from collections import Counter
    print(f"Total predictions: {len(predictions)}")
    print(f"Unique classes predicted: {set(predictions)}")
    print(f"Class distribution: {dict(Counter(predictions))}")

if __name__ == "__main__":
    main()

Epoch 1/20: 100%|██████████| 32/32 [01:14<00:00,  2.33s/it]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [1/20]
Train Loss: 1.7340
Train: Accuracy: 0.5180, Precision: 0.3816, Recall: 0.5180, F1: 0.4093
Val: Accuracy: 0.4950, Precision: 0.3877, Recall: 0.4950, F1: 0.4158
Learning Rate: 0.001
New best model saved with accuracy: 0.4950


Epoch 2/20: 100%|██████████| 32/32 [00:14<00:00,  2.23it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch [2/20]
Train Loss: 1.1380
Train: Accuracy: 0.5840, Precision: 0.6530, Recall: 0.5840, F1: 0.4964
Val: Accuracy: 0.5100, Precision: 0.6326, Recall: 0.5100, F1: 0.4602
Learning Rate: 0.001
New best model saved with accuracy: 0.5100


Epoch 3/20: 100%|██████████| 32/32 [00:15<00:00,  2.09it/s]


...

Epoch [19/20]
Train Loss: 0.0021
Train: Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000
Val: Accuracy: 0.7400, Precision: 0.7371, Recall: 0.7400, F1: 0.7320
Learning Rate: 0.0002621440000000001


Epoch 20/20: 100%|██████████| 32/32 [00:15<00:00,  2.04it/s]


Epoch [20/20]
Train Loss: 0.0020
Train: Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000
Val: Accuracy: 0.7450, Precision: 0.7423, Recall: 0.7450, F1: 0.7372
Learning Rate: 0.0002621440000000001
Initializing EvaluationDataset with root_dir: /workspace/data/VGG16_test_features/frontal_view/VGG16_features
Found 300 .npy files in /workspace/data/VGG16_test_features/frontal_view/VGG16_features


Evaluating: 100%|██████████| 19/19 [00:09<00:00,  2.07it/s]

Attempting to save results to /workspace/mamba2_test_results.csv
Number of file names: 300
Number of predictions: 300
Class names: ['Left Lane Change', 'Left Turn', 'Right Lane Change', 'Right Turn', 'Slow-Stop', 'Straight']
File should have been saved. Does it exist? True
File path: /workspace/mamba2_test_results.csv
File size: 15103 bytes
Testing completed. Results saved to mamba2_test_results.csv
Total predictions: 300
Unique classes predicted: {0, 1, 2, 3, 4, 5}
Class distribution: {3: 37, 5: 75, 2: 37, 1: 92, 0: 19, 4: 40}


In [20]:
test_dataset = EvaluationDataset('/workspace/data/VGG16_test_features/frontal_view/')
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Evaluate the model on the testing dataset
predictions, file_names = evaluate_model(model, test_loader, device)

# Define class names
class_names = ['Left Lane Change', 'Left Turn', 'Right Lane Change', 'Right Turn', 'Slow-Stop', 'Straight']

# Save results to CSV
save_results_to_csv(file_names, predictions, class_names, '/workspace/mamba2_test_results.csv')

print("Testing completed. Results saved to mamba2_test_results.csv")

Initializing EvaluationDataset with root_dir: /workspace/data/VGG16_test_features/frontal_view/VGG16_features
Found 0 .npy files in /workspace/data/VGG16_test_features/frontal_view/VGG16_features
Contents of the directory:
[]


NameError: name 'batch_size' is not defined